In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dc-metro-crime-data/dc_crime_add_vars.csv


In [2]:
import os;
os.listdir('/kaggle/input/dc-metro-crime-data')

['dc_crime_add_vars.csv']

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [4]:

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error


In [5]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/kaggle/input/dc-metro-crime-data/dc_crime_add_vars.csv')  # Use the CSV file name

# Display the first few rows of the dataset
print(df.head())

   Unnamed: 0  X            REPORT_DAT     SHIFT              OFFENSE  METHOD  \
0           1  1  8/31/2008 8:47:00 PM   EVENING          THEFT/OTHER  OTHERS   
1           2  2  9/1/2008 12:45:00 AM  MIDNIGHT  MOTOR VEHICLE THEFT  OTHERS   
2           3  3   9/1/2008 3:00:00 AM  MIDNIGHT          THEFT/OTHER  OTHERS   
3           4  4   9/9/2008 7:46:00 AM       DAY          THEFT/OTHER  OTHERS   
4           5  5  8/24/2008 8:00:00 PM   EVENING  MOTOR VEHICLE THEFT  OTHERS   

                                  BLOCK  DISTRICT    PSA  WARD  ...  year  \
0      3500 - 3599 BLOCK OF R STREET NW       2.0  206.0     2  ...  2008   
1      2000 - 2015 BLOCK OF O STREET NW       2.0  208.0     2  ...  2008   
2     700 - 799 BLOCK OF 11TH STREET NW       1.0  101.0     2  ...  2008   
3      1700 - 1799 BLOCK OF P STREET NW       2.0  208.0     2  ...  2008   
4  500 - 599 BLOCK OF INDIANA AVENUE NW       1.0  102.0     2  ...  2008   

  month day  hour minute  second    EW     NS     

In [6]:
y = df['month']
y

0         8
1         9
2         9
3         9
4         8
         ..
342862    3
342863    3
342864    3
342865    3
342866    3
Name: month, Length: 342867, dtype: int64

In [7]:
x = df.drop('month',axis=1)
x



,Unnamed: 0,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,...,date,year,day,hour,minute,second,EW,NS,quad,crimetype
0,1,1,8/31/2008 8:47:00 PM,EVENING,THEFT/OTHER,OTHERS,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,...,2008-08-31 20:47:00,2008,31,20,47,0,West,North,Northwest,Non-Violent
1,2,2,9/1/2008 12:45:00 AM,MIDNIGHT,MOTOR VEHICLE THEFT,OTHERS,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,...,2008-09-01 00:45:00,2008,1,0,45,0,West,North,Northwest,Non-Violent
2,3,3,9/1/2008 3:00:00 AM,MIDNIGHT,THEFT/OTHER,OTHERS,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,...,2008-09-01 03:00:00,2008,1,3,0,0,East,North,Northeast,Non-Violent
3,4,4,9/9/2008 7:46:00 AM,DAY,THEFT/OTHER,OTHERS,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,...,2008-09-09 07:46:00,2008,9,7,46,0,West,North,Northwest,Non-Violent
4,5,5,8/24/2008 8:00:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,...,2008-08-24 20:00:00,2008,24,20,0,0,East,North,Northeast,Non-Violent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342862,342863,342863,3/21/2008 1:00:00 PM,DAY,MOTOR VEHICLE THEFT,OTHERS,2200 - 2230 BLOCK OF 6TH STREET NW,3.0,305.0,1,...,2008-03-21 13:00:00,2008,21,13,0,0,East,North,Northeast,Non-Violent
342863,342864,342864,3/21/2008 9:08:00 PM,EVENING,THEFT/OTHER,OTHERS,3300 - 3341 BLOCK OF 16TH STREET NW,3.0,302.0,1,...,2008-03-21 21:08:00,2008,21,21,8,0,West,North,Northwest,Non-Violent
342864,342865,342865,3/21/2008 4:30:00 PM,EVENING,THEFT/OTHER,OTHERS,1500 - 1530 BLOCK OF PARK ROAD NW,3.0,302.0,1,...,2008-03-21 16:30:00,2008,21,16,30,0,East,North,Northeast,Non-Violent
342865,342866,342866,3/21/2008 5:15:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,3200 - 3299 BLOCK OF PARK PLACE NW,3.0,302.0,1,...,2008-03-21 17:15:00,2008,21,17,15,0,East,North,Northeast,Non-Violent


data splitting


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [9]:
x_train

,Unnamed: 0,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,...,date,year,day,hour,minute,second,EW,NS,quad,crimetype
218383,218384,218384,9/6/2011 5:30:00 PM,EVENING,ROBBERY,GUN,5000 - 5099 BLOCK OF ASTOR PLACE SE,6.0,604.0,7,...,2011-09-06 17:30:00,2011,6,17,30,0,East,South,Southeast,Violent
334328,334329,334329,11/23/2014 10:58:00 PM,EVENING,THEFT F/AUTO,OTHERS,3200 - 3299 BLOCK OF WARDER STREET NW,3.0,302.0,1,...,2014-11-23 22:58:00,2014,23,22,58,0,East,North,Northeast,Non-Violent
282650,282651,282651,8/10/2010 10:00:00 AM,DAY,THEFT/OTHER,OTHERS,1800 - 1899 BLOCK OF MARTIN LUTHER KING JR AVE...,7.0,701.0,8,...,2010-08-10 10:00:00,2010,10,10,0,0,East,South,Southeast,Non-Violent
247447,247448,247448,3/24/2015 6:58:00 AM,MIDNIGHT,SEX ABUSE,GUN,1 - 199 BLOCK OF 61ST STREET NE,6.0,608.0,7,...,2015-03-24 06:58:00,2015,24,6,58,0,East,North,Northeast,Violent
100400,100401,100401,4/24/2008 3:00:00 AM,MIDNIGHT,BURGLARY,OTHERS,5200 - 5499 BLOCK OF 16TH STREET NW,4.0,403.0,4,...,2008-04-24 03:00:00,2008,24,3,0,0,West,North,Northwest,Non-Violent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339799,339800,339800,8/9/2017 6:32:56 PM,EVENING,THEFT/OTHER,OTHERS,2300 - 2499 BLOCK OF CHAMPLAIN STREET NW,3.0,303.0,1,...,2017-08-09 18:32:56,2017,9,18,32,56,West,North,Northwest,Non-Violent
253799,253800,253800,11/23/2008 2:00:00 PM,DAY,THEFT/OTHER,OTHERS,3651 - 3799 BLOCK OF MARTIN LUTHER KING JR AVE...,7.0,707.0,8,...,2008-11-23 14:00:00,2008,23,14,0,0,East,South,Southeast,Non-Violent
210755,210756,210756,10/11/2008 2:50:00 PM,DAY,MOTOR VEHICLE THEFT,OTHERS,3000 - 3099 BLOCK OF ERIE STREET SE,6.0,606.0,7,...,2008-10-11 14:50:00,2008,11,14,50,0,East,South,Southeast,Non-Violent
56088,56089,56089,11/2/2015 10:25:51 AM,DAY,THEFT/OTHER,OTHERS,1100 - 1199 BLOCK OF 15TH STREET NW,2.0,207.0,2,...,2015-11-02 10:25:51,2015,2,10,25,51,East,North,Northeast,Non-Violent


In [10]:
x_test

,Unnamed: 0,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,...,date,year,day,hour,minute,second,EW,NS,quad,crimetype
132425,132426,132426,7/24/2015 7:07:00 PM,EVENING,THEFT/OTHER,OTHERS,1100 - 1121 BLOCK OF BLADENSBURG ROAD NE,5.0,507.0,5,...,2015-07-24 19:07:00,2015,24,19,7,0,East,North,Northeast,Non-Violent
2375,2376,2376,2/16/2009 6:27:00 PM,EVENING,THEFT/OTHER,OTHERS,1000 - 1099 BLOCK OF 26TH STREET NW,2.0,207.0,2,...,2009-02-16 18:27:00,2009,16,18,27,0,West,North,Northwest,Non-Violent
28447,28448,28448,9/14/2015 2:00:50 PM,DAY,THEFT/OTHER,OTHERS,3036 - 3099 BLOCK OF M STREET NW,2.0,206.0,2,...,2015-09-14 14:00:50,2015,14,14,0,50,West,North,Northwest,Non-Violent
265061,265062,265062,12/16/2012 6:36:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,1100 - 1199 BLOCK OF VARNEY STREET SE,7.0,706.0,8,...,2012-12-16 18:36:00,2012,16,18,36,0,East,South,Southeast,Non-Violent
21877,21878,21878,2/22/2014 8:49:00 AM,DAY,THEFT/OTHER,OTHERS,400 - 457 BLOCK OF MASSACHUSETTS AVENUE NW,1.0,102.0,2,...,2014-02-22 08:49:00,2014,22,8,49,0,East,North,Northeast,Non-Violent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96299,96300,96300,2/6/2016 9:22:21 PM,EVENING,THEFT/OTHER,OTHERS,5700 - 5899 BLOCK OF GEORGIA AVENUE NW,4.0,403.0,4,...,2016-02-06 21:22:21,2016,6,21,22,21,East,North,Northeast,Non-Violent
223795,223796,223796,9/23/2013 5:00:00 PM,EVENING,BURGLARY,OTHERS,2900 - 2999 BLOCK OF R STREET SE,6.0,607.0,7,...,2013-09-23 17:00:00,2013,23,17,0,0,East,South,Southeast,Non-Violent
57932,57933,57933,9/17/2016 5:56:14 PM,EVENING,THEFT F/AUTO,OTHERS,1400 - 1499 BLOCK OF 17TH STREET NW,2.0,208.0,2,...,2016-09-17 17:56:14,2016,17,17,56,14,West,North,Northwest,Non-Violent
331930,331931,331931,12/8/2012 2:20:00 AM,MIDNIGHT,ROBBERY,OTHERS,831 - 999 BLOCK OF EUCLID STREET NW,3.0,304.0,1,...,2012-12-08 02:20:00,2012,8,2,20,0,East,North,Northeast,Violent


In [11]:
y_train

218383     9
334328    11
282650     8
247447     3
100400     4
          ..
339799     8
253799    11
210755    10
56088     11
300552     8
Name: month, Length: 274293, dtype: int64

In [12]:
y_test

132425     7
2375       2
28447      9
265061    12
21877      2
          ..
96299      2
223795     9
57932      9
331930    12
48589      6
Name: month, Length: 68574, dtype: int64

Model Building

Linear Regression

In [13]:
# Check for any missing values
print(df.isnull().sum())

# Select numerical columns only
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df_numerical = df[numerical_cols]

Unnamed: 0                  0
X                           0
REPORT_DAT                  0
SHIFT                       0
OFFENSE                     0
METHOD                      0
BLOCK                       0
DISTRICT                  200
PSA                       251
WARD                        0
ANC                         0
NEIGHBORHOOD_CLUSTER     4705
BLOCK_GROUP              1091
CENSUS_TRACT             1091
VOTING_PRECINCT            84
CCN                         0
START_DATE                 13
END_DATE                11651
XBLOCK                      0
YBLOCK                      0
optional                    0
date                        0
year                        0
month                       0
day                         0
hour                        0
minute                      0
second                      0
EW                          0
NS                          0
quad                        0
crimetype                   0
dtype: int64


In [14]:
# Define your target variable and features
target = 'X'  # Replace with your actual target variable name
features = df_imputed.drop(columns=[target])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df_imputed[target], test_size=0.2, random_state=42)

# Instantiate the linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_test)

# Optionally, evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Print coefficients and intercept if needed
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

NameError: name 'df_imputed' is not defined